 ## Preparing Inputs
 The following code take in raster inputs,convert from ascii to tif, aggregtate and assign projection NAD83 CA Albers at 1km resolution

In [1]:
# PARALLEL PROCESSING VERSION 


import gdal
import numpy as np
import rasterio
import os
import osr
from rasterio.plot import show, show_hist
import gdal
import datetime
from pathlib import Path
import os.path
from joblib import Parallel, delayed
import multiprocessing
from pathlib import Path
import os.path     

def agg_proj_raster(metric, month, year, proj4, res, resample):
    
    # prep names 
    month_num = datetime.datetime.strptime(month,'%b').strftime('%m')
    in_name = r'C:\Users\mmann\Documents\ts-raster-example-data/BCMv8/'+metric+str(year)+month+'.asc'
    out_name = (r'C:\Users\mmann\Documents\ts-raster-example-data\BCMv8\resample/'+
                metric+'-'+str(year)+month_num+'.tif')
        
    if  os.path.exists(in_name) and not(os.path.exists(out_name)):    #avoid missing or completed
        # resamples, and assigns projection 
        drv = gdal.GetDriverByName('GTiff')
        ds_in = gdal.Open(in_name) 
        ds_in = gdal.Warp('',ds_in, resampleAlg =resample, format='MEM', xRes=res, yRes=res)  #resample
        name_out = (r'D:\HIST/'+metric+'-'+str(year)+month_num+'.tif') 
        ds_out = drv.CreateCopy(out_name, ds_in)  
        srs = osr.SpatialReference()
        srs.ImportFromProj4(proj4)
        ds_out.SetProjection(srs.ExportToWkt()) # assign projection
        ds_in = None  # clear memory 
        ds_out = None
    return 1


#setup parameters
res = 1000 
resample = 'average'
proj4 = '+proj=aea +lat_1=34 +lat_2=40.5 +lat_0=0 +lon_0=-120 +x_0=0 +y_0=-4000000 +ellps=GRS80 +datum=NAD83 +units=m +no_defs '     

# num of core for parallel 
num_cores = multiprocessing.cpu_count() -1
    

Parallel(n_jobs=num_cores) (delayed(agg_proj_raster) (metric = metric, 
                                              month = month,
                                              year= year,
                                              proj4=proj4, res=res, resample=resample) 
                    for metric in ['ppt','tmx','aet','cwd','pet','pck']    
                    for year in range(1949,2017)   
                    for month in ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'])
     


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
# NON PARALLEL VERSION 
# convert from ascii to tif, aggregtate and assign projection
# works in ts-raster & gdal_env_2.7v2 environment

import gdal
import numpy as np
#import rasterio
import os
import osr
import datetime
from pathlib import Path
import os.path
 
     
def agg_proj_raster(in_name, out_name, proj4, res, resample):
    # resamples, and assigns projection 
    drv = gdal.GetDriverByName('GTiff')
    ds_in = gdal.Open(in_name) 
    ds_in = gdal.Warp('',ds_in, resampleAlg =resample, format='MEM', xRes=res, yRes=res)  #resample
    #name_out = (r'D:\HIST/'+metric+'-'+str(year)+"%02d"+'.tif') %month_num
    ds_out = drv.CreateCopy(out_name, ds_in)  
    srs = osr.SpatialReference()
    srs.ImportFromProj4(proj4)
    ds_out.SetProjection(srs.ExportToWkt()) # assign projection
    ds_in = None  # clear memory 
    ds_out = None
    

#setup parameters
out_path=r'C:/Users/mmann/Documents/ts-raster-example-data/BCMv8/resample/'
res = 1000 
resample = 'average'
proj4 = '+proj=aea +lat_1=34 +lat_2=40.5 +lat_0=0 +lon_0=-120 +x_0=0 +y_0=-4000000 +ellps=GRS80 +datum=NAD83 +units=m +no_defs '     
    
for metric in ['ppt','tmx','aet','cwd','pet','pck']:    
    for year in range(1949,2017):  
         for month in ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']:    

            #set up parameters 
            month_num = datetime.datetime.strptime(month,'%b').strftime('%m')

            in_name = r'C:/Users/mmann/Documents/ts-raster-example-data/BCMv8/'+metric+str(year)+month+'.asc'
            out_name = (out_path+metric+'-'+str(year)+month_num+'.tif')  
            print(in_name)
            if  os.path.exists(in_name) and not(os.path.exists(out_name)):    #avoid missing or completed
                agg_proj_raster(in_name, out_name, proj4, res, resample)
     
